# Análisis de la criminalidad en California, USA

Este trabajo se corresponde a la **práctica 3** de la asignatura de **Minería de Datos**, sus autores son:

+ Javier García Sigüenza
+ Álvaro Navarro López-Menchero

## Objetivo

El objetivo es crear un modelo que pueda ayudar a determinar la importancia a asignar a un incidente, en base a la gravedad que se cree que tendrá este según una serie de datos aportados, para que se le pueda dar mayor o menor importancia. Una importancia baja corresponden a un incidente de gravedad baja, donde no se cree que se vayan a registrar heridos ni muertos. Uno de importancia media es donde se cree que se vaya a registrar heridos pero no muertos, y uno de importancia alta es donde la gravedad será alta, ya que el modelo ha predicho que habrá muertos o muertos y heridos. Como elemento extra también se ha buscado que el resultado obtenido sea explicable, para así evitar el funcionamiento de tipo caja negra. 

Por lo tanto el objetivo final es obtener un modelo capaz de ayudar de asistente a la hora de priorizar un incidente frente a otro en caso de falta de recursos en un cuerpo policial, a la vez que se consigue cierta explicabilidad del modelo para que este pueda ser de mayor confianza por parte del usuario.

El [dataset principal](https://www.kaggle.com/jameslko/gun-violence-data) es una recopilación de características presente en una serie de crímenes y fue filtrado para solo utilizar el Estado de **California**. Además, los datos fueron enriquecidos con otro [dataset extra](https://simplemaps.com/data/us-cities), que permitía conocer la población y densidad de población del lugar donde se cometió el crimen, así como completar datos relacionados con la latitud y longitud en algunos casos donde esta información no estaba disponible.

## Se importan las librerías y funciones necesarias

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from datetime import datetime
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sns

In [ ]:
from collections import Counter
#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

## Se realiza la carga de datos

In [ ]:
data_folder = "../input/crimen"
file_name = "gun-violence-data_01-2013_03-2018-california.csv"
path = os.path.join(data_folder,file_name)
data = pd.read_csv(path)
data.drop(columns = data.columns[0], inplace=True)

## Preprocesado de los datos

In [ ]:
# Creamos un backup inicial
data_backup = data.copy()
# Reindexamos por si hubiera algún problema con los datos cargados
data.reset_index(drop=True, inplace=True)

In [ ]:
# Mostramos la cantidad de datos
n_data = len(data)
print(f"Tenemos {n_data} filas de datos.")

La columna **incident_characteristics** tiene mucha información relevante, pero ya que el objetivo del proyecto es conseguir un predictor de la gravedad del indicente a priori esta sería problemática, ya que proporciona información del resultado del indicente. Por ello es eliminada.

In [ ]:
# Mostramos los datos
data.head()

Renombramos la columna **gun_stolen**, ya que este nombre se utilizará más adelante para una nueva columna.

In [ ]:
data.rename(columns={'gun_stolen':'raw_gun_stolen'}, inplace=True)

Eliminamos aquellas columnas que no vamos a utilizar

In [ ]:
cols_remove = ["incident_id", "incident_url", "source_url", "participant_age", "state", "sources", "incident_url_fields_missing", "location_description","notes", "participant_name", "participant_relationship", "participant_status", "gun_type"]
print(f"Vamos a borrar {len(cols_remove)} columnas de las {len(data.columns)} que hay en el dataframe")
data.drop(cols_remove, inplace=True, axis=1)

In [ ]:
print(f"Ahora tenemos {len(data.columns)} columnas")

In [ ]:
data.drop(columns = ["incident_characteristics"], inplace=True)
data.head()

Cambiamos los valores NaN en **participant_gender**, **participant_age_group**, **gun_stolen** y **participant_type** por el valor "-".

In [ ]:
data["participant_gender"] = data["participant_gender"].fillna("-")
data["participant_age_group"] = data["participant_age_group"].fillna("-")
data["raw_gun_stolen"] = data["raw_gun_stolen"].fillna("-")
data["participant_type"] = data["participant_type"].fillna("-")

Se va a utilizar la latitude y longitud para obtener clusteres y poder agrupar por zonas, es por ello que no podemos utilizar los casos donde no se ha definido este valor. Primero comprobamos que casos existen.

In [ ]:
len(data[data[["latitude"]].isna()["latitude"]])

In [ ]:
len(data[data[["longitude"]].isna()["longitude"]])

Como podemos ver existen el mismo caso de incidentes donde no se tiene la latitud que la longitud. En esta situación existen dos posibilidades, o eliminar estas entradas, o imputar los datos a partir de una segunda fuente de datos. Esta segunda opción fue la elegida, ya que al tener la ciudad se puede obtener unas coordenadas aproximadas.

In [ ]:
data_population = pd.read_csv("../input/crimen/uscities.csv")
data_population.head(3)

Filtramos los datos solo a California, ya que es el Estado que nos interesa.

In [ ]:
data_population = data_population[data_population["state_name"] == "California"]
data_population.head(3)

El conjunto de datos extra tiene otros datos interesante como son la población y la densidad de población, por lo que vamos a añadir también estos datos para enriquecer nuestro dataset.

In [ ]:
#Eliminamos " (county)" para poder unir los datos mas facilmente
#data["city_or_county"] = data["city_or_county"].replace({' \(county\)': ''}, regex=True)
#Iteramos las filas del DataFrame con iterrows() para realizarlo de forma más obtima a través
#del indice (index) y los datos de la fila (row)
for index, row in data.iterrows():
    #Imputamos la latitud
    if pd.isnull(row["latitude"]):
        #En ocasiones no se encuentra la ciudad y se lanza una excepción, en esos casos se deja el valor NaN
        #y la columna será borrada a posteriori
        try:
            lat = data_population[data_population["city"] == row["city_or_county"]]["lat"]
            data.at[index, "latitude"] = lat
        except:
            pass
    #Imputamos la longitud
    if pd.isnull(row["longitude"]):
        try:
            lng = data_population[data_population["city"] == row["city_or_county"]]["lng"]
            data.at[index, "longitude"] = lng
        except:
            pass
    #Unimos los datos de población
    try:
        population = data_population[data_population["city"] == row["city_or_county"].strip()]["population"]
        data.at[index, "population"] = population
    except:
        pass
    #Unimos los datos de densidad
    try:
        density = data_population[data_population["city"] == row["city_or_county"].strip()]["density"]
        data.at[index, "density"] = density
    except:
        pass
data.head()

In [ ]:
len(data[data[["population"]].isna()["population"]])

In [ ]:
len(data[data[["density"]].isna()["density"]])

In [ ]:
len(data[data[["latitude"]].isna()["latitude"]])

In [ ]:
len(data[data[["longitude"]].isna()["longitude"]])

In [ ]:
len(data)

Eliminamos todas las entradas que contengan nulos en la latitud, longitud, población o densidad de población.

In [ ]:
data = data[data["latitude"].notna()]
data = data[data["longitude"].notna()]
data = data[data["population"].notna()]
data = data[data["density"].notna()]

In [ ]:
len(data)

In [ ]:
# Reindexamos por si hubiera algún problema con los datos cargados
data.reset_index(drop=True, inplace=True)

A continuación se van a crear diversos flags relacionados con la edad, el sexo y elementos del crimen.

In [ ]:
# Columnas que creamos
data["number_implicates"] = 0
data["exist_suspect"] = False
data["male_involved"] = False
data["female_involved"] = False
data["gun_stolen"] = False
data["gun_involved"] = False
data["child_involved"] = False
data["teen_involved"] = False
data["adult_involved"] = False

Añadimos campos de fecha interesantes para poder trabajar con más información

In [ ]:
# Creamos las columnas para la información
data["year"] = "initial_value"
data["id_month"] = "initial_value"
data["month"] = "initial_value"
data["day"] = "initial_value"

In [ ]:
# Creamos el diccionario para pasar de id del mes al propio mes
dict_index_month = {1: "january", 2: "february", 3: "march", 4: "april", 
                    5: "may", 6: "june", 7: "july", 8: "august", 
                    9: "september", 10: "october", 11: "november", 12: "december"}

In [ ]:
# Recorremos cada fila del dataframe 
for index, row in data.iterrows():
    # Cada fila es tipo str
    date_tmp = row["date"]
    data.at[index, "year"] = pd.to_datetime(date_tmp, format='%Y-%m-%d').year
    data.at[index, "id_month"] = pd.to_datetime(date_tmp, format='%Y-%m-%d').month
    data.at[index, "month"] = dict_index_month[data.at[index, "id_month"]]
    data.at[index, "day"] = pd.to_datetime(date_tmp, format='%Y-%m-%d').day

In [ ]:
data[["date", "year", "month", "id_month", "day"]].head()

Primero tratamos las variables relacionadas con la edad y el sexo.

In [ ]:
#Iteramos las filas del DataFrame con iterrows() para realizarlo de forma más obtima a través
#del indice (index) y los datos de la fila (row)
for index, row in data.iterrows():
    #Comprobamos la presencia de hombres
    if "male" in row["participant_gender"].lower():
        data.at[index, "male_involved"] = True
    #Comprobamos la presencia de mujeres
    if "female" in row["participant_gender"].lower():
        data.at[index, "female_involved"] = True
    #Comprobamos la presencia de niños
    if "child" in row["participant_age_group"].lower():
        data.at[index, "child_involved"] = True
    #Comprobamos la presencia de adolescentes
    if "teen" in row["participant_age_group"].lower():
        data.at[index, "teen_involved"] = True
    #Comprobamos la presencia de adultos
    if "adult" in row["participant_age_group"].lower():
        data.at[index, "adult_involved"] = True
data[["participant_gender", "participant_age_group", "male_involved", "female_involved", "child_involved", \
     "teen_involved", "adult_involved"]].head()

Ahora tratamos las variables relacionadas con el crimen.

In [ ]:
for index, row in data.iterrows():
    #Comprobamos si ha habido armas implicada en el incidente
    if row["raw_gun_stolen"] != "-":
        data.at[index, "gun_involved"] = True
    #Comprobamos si hay armas robadas implicadas en el incidente
    all_guns = row["raw_gun_stolen"].split("||")
    stolen_gun = False
    for gun_info in all_guns:
        if "stolen" in gun_info.lower():
            stolen_gun = True
    if stolen_gun:
        data.at[index, "gun_stolen"] = True
    #Comprobamos el número de implicados en el incidente
    if row["participant_type"] != "-":
        all_involveds = row["participant_type"].split("||")
        data.at[index, "number_implicates"] = len(all_involveds)
    # Comprobamos si se han identificado sospechosos
    if "suspect" in row["participant_type"].lower():
        data.at[index, "exist_suspect"] = True
        

In [ ]:
data[["gun_involved", "gun_stolen", "number_implicates", "exist_suspect"]].head()

### Se añade la variable salida

Se clsifica en:
- 0: Importancia baja -> sin heridos ni muertos
- 1: Importancia media -> con heridos, pero sin muertos
- 2: Importancia alta -> con muertos

In [ ]:
def get_class(n_killed, n_injured):
    if n_killed==0 and n_injured==0:
        return 0
    elif n_killed==0 and n_injured>0:
        return 1
    elif n_killed>0:
        return 2
    else:
        raise Exception("Valores incorrectos tratando la variable respuesta")

In [ ]:
# Se utilizará n_killed (fallecidos) y n_injured (heridos) para asignar la variable
# Declaramos las columnas
y_column = "severity"
# Inicializamos la clase al valor 0: sin heridos ni muertos
data[y_column] = 0
# Recorremos cada fila del dataframe y asignamos el valor adecuado
n_rows = len(data)
for index, row in data.iterrows():
    y_value = get_class(row["n_killed"], row["n_injured"]) 
    data.at[index, y_column] = y_value

# Por último, como la clase ha sido generada mediante dos columnas
# habrá dependencia lineal, borramos dichas variables
data.drop(["n_killed", "n_injured"], inplace=True, axis=1)
data.head()

### Balanceo de los datos

Primero de todo, mostramos los datos relativos a cada clase para poder observar la proporción entre ellas.

In [ ]:
# obtener algunas estadísticas sobre los datos
categories = sorted(data[y_column].unique(), reverse=False)
hist= Counter(data[y_column]) 
print(f'Total de instancias -> {data.shape[0]}')
distribution_classes = data[y_column].value_counts(normalize=True) * 100
print(f'Distribución de clases -> {distribution_classes.to_list()}')

print(f'Categorías -> {categories}')
print(f'Categoría del comentario -> {data[y_column][0]}')

n_classes = len(categories)
print(f"Tenemos {n_classes} clases en el dataset: {categories}")

categories_names = ["Importancia baja","Importancia media", "Importancia alta"]# 0, 1, 2
print(f"Dichas {n_classes} clases se corresponden a: {categories_names}")

In [ ]:
# Gráfico de barras
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

In [ ]:
# Gráfico de tarta
amounts = []
colors = ["purple", "blue", "red"]
for cat in categories:
    amount_tmp = len(data.loc[data[y_column]==cat,])
    amounts.append(amount_tmp)
plt.pie(amounts, labels=categories_names, startangle = 90, autopct='%1.2f%%', shadow=True, colors=colors)
plt.show()

Se observa que las proporciones de cada clase son parecidas por lo que **no es necesario realizar un balanceo explícito de los datos**.

In [ ]:
data.head()

### Clusters

Se van a crear dos clusters diferentes, para dividir los diferentes casos en base a las áreas geográficas donde tuvieron lugar, así como en base a la densidad de población de la ciudad donde tiene lugar el incidente.

In [ ]:
import numpy as np
latitude = data["latitude"].to_list()
longitude = data["longitude"].to_list()
cluster_data = list(zip(latitude, longitude))
sum_of_squared_distances = []
num_clusters = range(1,10)
for k in num_clusters:
    kmeans = KMeans(n_clusters=k)
    kmeans = kmeans.fit(cluster_data)
    sum_of_squared_distances.append(kmeans.inertia_)

plt.plot(num_clusters, sum_of_squared_distances, 'bx-')
plt.xlabel('Valor de k')
plt.ylabel('Suma de las distancias al cuadrado')
plt.title('Método del codo para buscar una k optima')
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

num_clusters = range(2,10)
for k in num_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(cluster_data) + (k + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=k, random_state=10)
    cluster_labels = clusterer.fit_predict(cluster_data)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(cluster_data, cluster_labels)
    print("For n_clusters =", k,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(cluster_data, cluster_labels)

    y_lower = 10
    for i in range(k):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / k)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / k)
    ax2.scatter(np.array(cluster_data)[:, 0], np.array(cluster_data)[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % k),
                 fontsize=14, fontweight='bold')

plt.show()


Como se puede observar por ambos criterios el mejor es con k=2.

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans = kmeans.fit(cluster_data)
kmeans.labels_
data["area_cluster"] = kmeans.labels_

In [ ]:
#One hot del cluster
pop_dens_cluster = pd.get_dummies(data["area_cluster"])
pop_dens_cluster.columns = ["area_0", "area_1"]
data = pd.concat([data, pop_dens_cluster], axis=1)
data.head()

Ahora vamos a realizar un segundo cluster para la densidad de población.

In [ ]:
import numpy as np
latitude = data["density"].to_list()
cluster_data = np.reshape(np.array(latitude), (len(latitude), -1))
sum_of_squared_distances = []
num_clusters = range(1,10)
for k in num_clusters:
    kmeans = KMeans(n_clusters=k)
    kmeans = kmeans.fit(cluster_data)
    sum_of_squared_distances.append(kmeans.inertia_)

plt.plot(num_clusters, sum_of_squared_distances, 'bx-')
plt.xlabel('Valor de k')
plt.ylabel('Suma de las distancias al cuadrado')
plt.title('Método del codo para buscar una k optima')
plt.show()

El mejor número de clusters parece ser k=5.

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans = kmeans.fit(cluster_data)
kmeans.labels_
data["pop_dens_cluster"] = kmeans.labels_

In [ ]:
sns.scatterplot(data=data, x="latitude", y="longitude", hue="pop_dens_cluster")
plt.show()

In [ ]:
#One hot del cluster
pop_dens_cluster = pd.get_dummies(data["pop_dens_cluster"])
pop_dens_cluster.columns = ["pop_dens_0", "pop_dens_1", "pop_dens_2", "pop_dens_3", "pop_dens_4"]
data = pd.concat([data, pop_dens_cluster], axis=1)
data.head()

In [ ]:
data.columns

## Últimos pasos

Por último, borramos columnas inservivbles y comprobamos la distribución por años de los datos.

In [ ]:
# Borramos las columnas que no necesitaremos en adelante
list_remove_columns = ["address", "congressional_district", "raw_gun_stolen", "n_guns_involved", "participant_age_group", "participant_gender", "participant_type", "state_house_district", "state_senate_district", ]
data.drop(columns = list_remove_columns, inplace=True)

In [ ]:
# Lista con los años
list_years = sorted(list(set(data["year"])))
# Guardamos la cantidad para cada año
n_crimes = {}
for year in list_years:
    n_crimes[year] = len(data.loc[data["year"] == year,])
# Creamos las variables y el dataframe
years = list(n_crimes.keys())
crimes = list(n_crimes.values())
df_n_crimes = pd.DataFrame({'year': years, 'n_crimes': crimes})

In [ ]:
df_n_crimes

Se observa que apenas hay isntancias de 2013, debido a esto se procede a eliminarlos para que no estén entorpeciendo las próximas tareas.

In [ ]:
print(f"Inicialmente hay {len(data)} filas de información")
# Creamos un backup
data_backup = data.copy()
# Guardamos los índices a borrar
list_index_2013 = list(data.loc[data["year"] == 2013,].index)
# Borramos los datos
data = data.drop(list_index_2013)
# Reseteamos los índices
data = data.reset_index()
print(f"Tras eliminar los datos de 2013 hay {len(data)} filas de información")
print(f"Se han eliminado {abs(len(data)-len(data_backup))} filas")

## Exportar

In [ ]:
data.to_csv("preprocessed_data.csv", encoding='utf-8-sig')
data.to_json("preprocessed_data.json")

# Visualiazación


## Importamos las librerías y funciones necesarias

In [ ]:
import pandas as pd
import os
import math

In [ ]:
from collections import Counter
#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Carga los datos

In [ ]:
data_folder = ""
file_name = "preprocessed_data.csv"
path = os.path.join(data_folder,file_name)
data = pd.read_csv(path)
# Al cargar el csv se añade un índice nuevo inservible: lo eliminamos
data.drop(columns = data.columns[0], inplace=True)

In [ ]:
data.head()

## Funciones auxiliares

In [ ]:
# Funcies que dada una lista devuelve un dataframe y el diccionario que lo compone
# Dado el dataframe y la columna
def get_dic_df_data_value(data, column):
    d = {}
    list_values = sorted(list(set(data[column])))
    for value in list_values:
        d[value] = len(data.loc[data[column] == value,])
    keys = list(d.keys())
    values = list(d.values())
    column_name = "n_crimes_" + column
    df = pd.DataFrame({column: keys, column_name: values})
    return d, df
# Dado el dataframe y la lista de columnas
def get_dic_df_data_column(data, list_columns, name):
    d = {}
    for value in list_columns:
        d[value] = len(data.loc[data[value] == True,])
    keys = list(d.keys())
    values = list(d.values())
    column_name = "n_crimes_" + name
    df = pd.DataFrame({name: keys, column_name: values})
    return d, df
# Función devuelve la carpeta donde guardar las figuras
def check_folder(file_name, folder_name, sup_folder="other_data"):
    # sub_folder = os.path.join(folder_name, os.path.join(sup_folder, file_name)) # <- varias carpetas
    sub_folder = folder_name
    if not os.path.exists(sub_folder):
        # print(f"Creamos la carpeta: {sub_folder}")
        os.makedirs(sub_folder)
    return sub_folder

## Estadísticas descriptivas

### Datos del dataframe

Aquí se muestran los datos del proppio conjunto, todas las columnas y las primeras filas.

In [ ]:
data.head()

Ahora, se muestran los tipos del conjunto de datos, en concreto, el tipo de cada columna.

In [ ]:
data.info()

En esta sección, se muestran características de las columnas, número máximo, mínimo, etc. Esto es de suma importancia para identificar columnas relevantes a la hroa de visualizar.

In [ ]:
data.describe()

Como se puede observar en esta descripción de las columnas, hay datos que no tiene sentido analizar en profundidad, por ello se seleccionan las de mayor importancia para el estudio del trabajo.

### Correlaciones

Aquí se muestra, para las columnas más relevantes, el mapa de calor con las relaciones entre las distitnas variables.

In [ ]:
columns_sd = ['population', 'density', 'number_implicates', 'exist_suspect',
       'male_involved', 'female_involved', 'gun_stolen', 'gun_involved',
       'child_involved', 'teen_involved', 'adult_involved', 'year']

In [ ]:
# Mostramos el mapa de calor entre las distintas variables del conjunto de datos
plt.figure(figsize=(14,10))
sns.heatmap(data[columns_sd].corr('spearman'), annot=True, linewidth=3)
plt.yticks(rotation=0)
plt.xticks(rotation=90)
# Obtenemos la carpeta donde guardar el gráfico
sub_folder = check_folder("heat_map", "figs")
# Exportamos a JPG
fig_jpg = os.path.join(sub_folder, "heat_map" + '.jpg')
plt.savefig(fig_jpg)
# Exportamos a PDF
# fig_pdf = os.path.join(sub_folder, "heat_map" + '.pdf')
# plt.savefig(fig_pdf)
# Mostramos el plot
plt.show()

Se pueden extraer conclusioens interesantes de estas relaciones, algunas son:

+ La población y la densidad están (positivamente) fuertemente relacionadas.
+ Una fuerte conexión entre la presencia de adultos y de hombres.
+ La presencia de adultos va ligada a que no estén presentes adolescentes o niños.
+ El año del accidente tiene una relación muy fuerte y positiva con la presencia de armas.
+ Relación fuerte y positiva entre el número de implicados y la detección de sospechosos.

Dado que la relación más importante es, a priori, la de la población y la densidad de la población, se expone aquí la relación entre ambas.

In [ ]:
# Como la correlación de mayor interés ha sido la de población y densidad 
# la mostramos de nuevo únicamente entre ellas con relplot
sns.relplot(x='population', y='density', data=data)
# Obtenemos la carpeta donde guardar el gráfico
sub_folder = check_folder("rel_poblation_density", "figs")
# Exportamos a JPG
fig_jpg = os.path.join(sub_folder, "rel_poblation_density" + '.jpg')
plt.savefig(fig_jpg)
# Exportamos a PDF
# fig_pdf = os.path.join(sub_folder, "rel_poblation_density" + '.pdf')
# plt.savefig(fig_pdf)
# Mostramos el plot
plt.show()

Se observa algún valor atípico al aumentar la población.

### Boxplot


Ahora, se van a mostrar la dispersión de las variables numéricas, en concreto, se mostrará la dispersión del número de implicados, la población y la densidad de la población.

In [ ]:
# Número de implicados
plt.figure(figsize=(10, 8))
sns.boxplot(data=data["number_implicates"])
plt.xlabel('Crímen')
plt.ylabel('Número de implicados')
# Obtenemos la carpeta donde guardar el gráfico
sub_folder = check_folder("box_plot_n_implicates", "figs")
# Exportamos a JPG
fig_jpg = os.path.join(sub_folder, "box_plot_n_implicates" + '.jpg')
plt.savefig(fig_jpg)
# Exportamos a PDF
# fig_pdf = os.path.join(sub_folder, "box_plot_n_implicates" + '.pdf')
# plt.savefig(fig_pdf)
# Mostramos el plot
plt.show()

In [ ]:
# Población
plt.figure(figsize=(10, 8))
sns.boxplot(data=data["population"])
plt.xlabel('Crímen')
plt.ylabel('Población')
# Obtenemos la carpeta donde guardar el gráfico
sub_folder = check_folder("box_plot_population", "figs")
# Exportamos a JPG
fig_jpg = os.path.join(sub_folder, "box_plot_population" + '.jpg')
plt.savefig(fig_jpg)
# Exportamos a PDF
# fig_pdf = os.path.join(sub_folder, "box_plot_population" + '.pdf')
# plt.savefig(fig_pdf)
# Mostramos el plot
plt.show()

In [ ]:
# Densidad
plt.figure(figsize=(10, 8))
sns.boxplot(data=data["density"])
plt.xlabel('Crímen')
plt.ylabel('Densidad')
# Obtenemos la carpeta donde guardar el gráfico
sub_folder = check_folder("box_plot_density", "figs")
# Exportamos a JPG
fig_jpg = os.path.join(sub_folder, "box_plot_density" + '.jpg')
plt.savefig(fig_jpg)
# Exportamos a PDF
# fig_pdf = os.path.join(sub_folder, "box_plot_density" + '.pdf')
# plt.savefig(fig_pdf)
# Mostramos el plot
plt.show()

Quien **menos dispersión** tiene es la **densidad**, hay una **gran presencia de valores atípicos** que se salen del cuarto cuartil.

## Gráficas

Se muestran gráficas para cada una de las columnas de relevancia en el dataset, con agrupaciones y evoluciones temporales. Tanto para el conjunto total como para cada subconjunto en función de la severidad.

In [ ]:
print(f"El total de crímenes registrados es de {len(data)}")

In [ ]:
n_crimes_cluster, df_n_crimes_cluster = get_dic_df_data_value(data, "area_cluster")
df_n_crimes_cluster.head()

In [ ]:
tam_fig = 14
xticks_position = range(math.floor(min(data["year"])), math.ceil(max(data["year"]))+1)

# Gráficas de implicados agrupados por año
def show_crimes_year(data, sup_folder):
    plt.figure(figsize=(14,6))
    n_crimes_year, df_n_crimes_year = get_dic_df_data_value(data, "year")
    #df_n_crimes_year.plot(x="year",y="n_crimes_year", fontsize=15, kind="bar")
    ax = sns.barplot(x="year", y="n_crimes_year", data=df_n_crimes_year)
    ax.set_title("Cantidad de crímenes por año")
    ax.set_xlabel("Año")
    ax.set_ylabel('Cantidad de crímenes')
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("crimes_year", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_crimes_year" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_crimes_year" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()
    
# Gráficas de crímenes agrupados por el cluster
def show_crimes_cluster(data, sup_folder):
    n_crimes_cluster, df_n_crimes_cluster = get_dic_df_data_value(data, "area_cluster")
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(tam_fig,6))
    fig.tight_layout(pad=6.0)
    # Subplot 1
    df_n_crimes_cluster.plot(ax=axes[0], x="area_cluster",y=["n_crimes_area_cluster"], fontsize=15, kind="bar")
    axes[0].set_title("Cluster en base al área geográfica")
    axes[0].tick_params('x', labelrotation=0)
    axes[0].set_xlabel("Área")
    axes[0].set_ylabel('Cantidad de crímenes')
    axes[0].legend(["Cantidad de crímenes"], loc="upper right")
    # Subplot 2
    n_crimes_cluster, df_n_crimes_cluster = get_dic_df_data_value(data, "pop_dens_cluster")
    df_n_crimes_cluster.plot(ax=axes[1], x="pop_dens_cluster",y=["n_crimes_pop_dens_cluster"], fontsize=15, kind="bar")
    axes[1].set_title("Cluster en base a la densidad de la población")
    axes[1].tick_params('x', labelrotation=0)
    axes[1].set_xlabel("Densidad de población")
    axes[1].set_ylabel('Crímenes')
    axes[1].legend(["Cantidad de crímenes"], loc="upper right")
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("crimes_clusters", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_crimes_cluster" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_crimes_cluster" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()
    
# Gráficas de implicados en función del año
def show_implicates_year(data, sup_folder):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(tam_fig,6))
    fig.tight_layout(pad=6.0)
    # Subplot 1
    sns.lineplot(ax=axes[0], data=data.groupby(by='year')['number_implicates'].sum(), label="Suma total de los implicados")
    axes[0].set_xlabel("Año", size=15)
    axes[0].set_ylabel("Suma total", size=15)
    axes[0].tick_params(labelsize=15)
    axes[0].set_title("Evolución temporal del número total de implicados", size=15)
    axes[0].set_xticks(xticks_position)
    # Subplot 2
    sns.lineplot(ax=axes[1], data=data.groupby(by='year')['number_implicates'].mean(), label="Valor medio de los implicados")
    axes[1].set_xlabel("Año", size=15)
    axes[1].set_ylabel("Valor medio", size=15)
    axes[1].tick_params(labelsize=15)
    axes[1].set_title("Evolución temporal del número medio de implicados", size=15)
    axes[1].set_xticks(xticks_position)
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("implicates_year", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_implicates_year" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_implicates_year" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()
    
    
# Gráficas del género
def show_genre(data, sup_folder):
    plt.figure(figsize=(tam_fig, 10))
    sns.lineplot(data=data.groupby(by='year')['male_involved'].sum(), label="Cantidad de hombres identifciados")
    sns.lineplot(data=data.groupby(by='year')['female_involved'].sum(), label="Cantidad de mujeres identificadas")
    # Total
    sns.lineplot(data=data.groupby(by='year')['date'].count(), linestyle = "--", label="Total")
    plt.xlabel("Año", size=20)
    plt.ylabel("Cantidad", size=20)
    plt.tick_params(labelsize=20)
    plt.legend(loc=2,prop={'size':20})
    plt.title("Evolución temporal de los hombres y las mujeres implicadas en crímenes", size=15)
    plt.xticks(xticks_position)
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("genre", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_genre" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_genre" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()
    
    
# Gráficas de la edad
def show_age(data, sup_folder):
    plt.figure(figsize=(tam_fig, 10))
    sns.lineplot(data=data.groupby(by='year')['child_involved'].sum(), label="Cantidad de niños indentificados")
    sns.lineplot(data=data.groupby(by='year')['teen_involved'].sum(), label="Cantidad de adolescentes indentificados")
    sns.lineplot(data=data.groupby(by='year')['adult_involved'].sum(), label="Cantidad de adultos indentificados")
    # Total
    sns.lineplot(data=data.groupby(by='year')['date'].count(), linestyle = "--", label="Total")
    plt.xlabel("Año", size=20)
    plt.ylabel("Cantidad", size=20)
    plt.tick_params(labelsize=20)
    plt.legend(loc=2,prop={'size':20})
    plt.title("Evolución temporal de los adultos, adolescentes y niños identificados en crímenes", size=15)
    plt.xticks(xticks_position)
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("age", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_age" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_age" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()
    
    
# Gráficas de las armas
def show_guns(data, sup_folder):
    plt.figure(figsize=(tam_fig, 10))
    sns.lineplot(data=data.groupby(by='year')['gun_stolen'].sum(), label="Cantidad de armas robadas")
    sns.lineplot(data=data.groupby(by='year')['gun_involved'].sum(), label="Cantidad de armas implicadas")
    # Total
    sns.lineplot(data=data.groupby(by='year')['date'].count(), linestyle = "--", label="Total")
    plt.xlabel("Año", size=20)
    plt.ylabel("Cantidad", size=20)
    plt.tick_params(labelsize=20)
    plt.legend(loc=2,prop={'size':20})
    plt.title("Evolución temporal de las armas y armas robadas indetificadas en crímenes", size=15)
    plt.xticks(xticks_position)
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("guns", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_guns" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_guns" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()
    
    
# Sospechosos
def show_suspects(data, sup_folder):
    plt.figure(figsize=(tam_fig, 10))
    sns.lineplot(data=data.groupby(by='year')['exist_suspect'].sum(), label="Cantidad de sospechosos detectados")
    # Total
    sns.lineplot(data=data.groupby(by='year')['date'].count(), linestyle = "--", label="Total")
    plt.xlabel("Año", size=20)
    plt.ylabel("Cantidad", size=20)
    plt.tick_params(labelsize=20)
    plt.legend(loc=2,prop={'size':20})
    plt.title("Evolución temporal de los sospechosos indetificados en crímenes", size=15)
    plt.xticks(xticks_position)
    # Obtenemos la carpeta donde guardar el gráfico
    sub_folder = check_folder("suspects", "figs", sup_folder)
    # Exportamos a JPG
    fig_jpg = os.path.join(sub_folder, sup_folder+"_suspects" + '.jpg')
    plt.savefig(fig_jpg)
    # Exportamos a PDF
    # fig_pdf = os.path.join(sub_folder, sup_folder+"_suspects" + '.pdf')
    # plt.savefig(fig_pdf)
    # Mostramos el plot
    plt.show()


In [ ]:
# Cantidad de crímentes
print("Mostramos las gráficas del número de crímenes por año")
show_crimes_year(data, "data")

El año con menos críemenes detectados es, con diferencia, 2018.

In [ ]:
# Cluster geográfico
print("Agrupamos los crímenes por los clústers")
show_crimes_cluster(data, "data")

Aquí tenemos dos clusters distintos
+ número de **crímenes agrupados por cluster geográfico**: el área 1 tiene menos cantidad de crímenes.
+ número de **crímenes agrupados por cluster poblacional**: en una densidad media es donde menos crímenes suceden.

In [ ]:
# Implicados
print("Mostramos las gráficas de los implicados")
show_implicates_year(data, "data")

Se observa que la **suma** de crímenes se **reduce** en **2018**, pero la **media aumenta**. Eso se debe a que, como se ha comentado en la primera gráfica aquí expuesta, la cantidad de datos es mucho menor en 2018.

In [ ]:
# Género
print("Mostramos las gráficas en función del género")
show_genre(data, "data")

En una **gran proporción** de los crímenes hay implicados **hombres**. Sin embargo, en una **proporción** muy **reducida** lo están las mujeres.

In [ ]:
# Edad
print("Mostramos las gráficas en función de la edad")
show_age(data, "data")

La presencia de **adultos** es muy elevada, a diferencia de niños y adolescentes.

In [ ]:
# Armas involucradas y robadas
print("Mostramos las armas involucradas y si fueron robadas")
show_guns(data, "data")

La cantidad de armas robadas implicadas es muy baja, pero la presencia *per se* de armas es muy alta, de hecho, están implicadas en el 100% de casos de 2018.

In [ ]:
# Sospechosos detectados
print("Mostramos las gráficas de los sospechosos detectados")
show_suspects(data, "data")

En cuanto a los **sospechosos** detectados, la proporción de sospechosos detectados es, apróximadamente del **70%**.

In [ ]:
# Creamos una función que dado un dataframe obtengamos las gráficas importantes
def show_graphics(data, sup_folder):
    # Cantidad de crímentes
    print("Mostramos las gráficas del número de crímenes por año")
    show_crimes_year(data, sup_folder)
    # Clusters
    print("Agrupamos los crímenes por los clústers")
    show_crimes_cluster(data, sup_folder)
    # Implicados
    print("Mostramos las gráficas de los implicados")
    show_implicates_year(data, sup_folder)
    # Género
    print("Mostramos las gráficas en función del género")
    show_genre(data, sup_folder)
    # Edad
    print("Mostramos las gráficas en función de la edad")
    show_age(data, sup_folder)
    # Armas involucradas y robadas
    print("Mostramos las armas involucradas y si fueron robadas")
    show_guns(data, sup_folder)
    # Sospechosos detectados
    print("Mostramos las gráficas de los sospechosos detectados")
    show_suspects(data, sup_folder)


### Separamos los datos por su valor de clase

Realizamos gráficas para cada valor de clase, tendremos 3 dataframes.

**Clases**(severity):
+ **0** - Importancia **baja**
+ **1** - Importancia **media**
+ **2** - Importancia **alta**

#### Clase 0

Aquí se van a mostrar las gráficas para el conjunto de datos de importancia baja(clase 0).

In [ ]:
df_0 = data.loc[data["severity"]==0,].copy()
show_graphics(df_0, "df_0")

Para la **clase 0** se observa que se es muy pareja a las gráficas del dataframe completo, pero hay algún matiz en el número **total** y **medio** de los **implicados**. En esta ocasión la media no suba de manera tan líneal, sino que de 2016 a 2017 ese crecimiento acelera negativamente.

También se aprecia que el número de sospechosos detectados es mucho más alto en relación a la cantidad total de crímenes.

#### Clase 1

Aquí se van a mostrar las gráficas para el conjunto de datos de importancia media(clase 1).

In [ ]:
df_1 = data.loc[data["severity"]==1,].copy()
show_graphics(df_1, "df_1")

Para la **clase 1** se observa que, pese a ser muy parecida al dataframe completo, el mayor matiz ocurre con los **sopechosos detectados**.

El **sospechosos** detectados: la proporción de sospechosos detectados es, apróximadamente, algo inferior al 50%.

#### Clase 2

Aquí se van a mostrar las gráficas para el conjunto de datos de importancia alta(clase 2).

In [ ]:
df_2 = data.loc[data["severity"]==2,].copy()
show_graphics(df_2, "df_2")

Para la **clase 2** se observa que se parece sobre todo a la **clase 1**, en este caso el crecimiento de la **media** de los **implicados** es aún mayor.

## Conclusiones de la visualización

Se ha observado que todas las gráficas tienen comportamientos similares, la **clase 0** es la que más se parece al **original**. En cambio, la **clase 1** se asemeja más a la **clase 2**, esto puede corroborarse, por ejemplo, apreciando que, pese a que el porcentaje de hombres detectados siempre es muy alto, en las clases 1 y 2 es aún más alto.

Entre la clase **0** y las clases **1** y **2** (que son muy similares) hay algunas diferencia que podría provocar que ocurran diferencias a la hora de la predicción, dificultando la diferenciación entre estas dos últimas clases.

Se ha descartado automatizar el 100% del código para tener cada subdataframe separado y así poder comentarlo. De la misma manera las primeras gráficas (relativas al conjunto de datos completo) no usan la función `show_graphics` para poder comentar cada gráfica de manera aislada.

# Predicción

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from datetime import datetime
from sklearn.cluster import KMeans
import numpy as np
import time
#SKLearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
#Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

In [ ]:
# Try except para instalarlo si fuera necesario
try:
    import shap
except:
    !pip install shap
    import shap

Comenzamos el apartado de predicción estableciendo la **seed** a utilizar para intentar que los resultados sean los más repetibles posible.

In [ ]:
np.random.seed(0)

Primero cargamos los datos y mostramos que han sido cargados correctamente.

In [ ]:
data = pd.read_json('data/preprocessed_data.json')
data.drop(columns=["index"], inplace=True)
data.head()

In [ ]:
data.columns

Antes de comenzar a realizar las predicciones escalamos los datos y definimos una serie de funciones auxiliares.

In [ ]:
#Seleccionamos solo las variables que se van a usar para el entrenamiento
train_variables = ["population", "density", "number_implicates", "exist_suspect", "male_involved", \
                   "female_involved", "gun_stolen", "gun_involved", "child_involved", \
                   "teen_involved", "adult_involved", "area_0", "area_1", "pop_dens_0", \
                   "pop_dens_1", "pop_dens_2", "pop_dens_3", "pop_dens_4"]
x = data[train_variables].copy()
y = data["severity"].copy()
scaler = MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
x.head()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión (normalizada)")
    else:
        print('Matriz de confusión (sin normalizar)')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
    
def calculate_confusion_matrix(y_test, y_pred):
    #            Predict label
    #            _________
    #            |        |
    # True label |        |
    #            |________|
    #
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    plot_confusion_matrix(conf_matrix, classes=['Baja', 'Media', 'Alta'],
                          title='Confusion matrix, without normalization')
    plt.show()
    

def train_with_CV_class(model, x, y, n_splits = 5, number_decimals = 2, show_confusion_matrix = True):
    #Declaramos la función que generará las particiones del CV
    cv = KFold(n_splits=n_splits) 
    print(f"Se entrenará el modelo utilizando CV con k = {n_splits}")
    #Variable para acumular la media de los scores sobre los diferentes folds
    mean_score = 0.0
    iteration = 0
    #A la hora de calcula la matriz de confusión se podría utilizar el último modelo entrenado en las
    #particiones de K-Fold pero otra alternativa es obtener la predicción de y_test y los valores
    #reales de y_test y acumularlos para mostrar al final una matriz de confusión que represente el conjunto
    #de modelos entrenados sobre las diferentes particiones
    y_test_hist = np.array([])
    y_pred_hist = np.array([])
    for train_index, test_index in cv.split(x):
        #Generamos el conjunto de entramiento para la partición K
        x_train, y_train = x.iloc[train_index], y.iloc[train_index]
        x_test, y_test = x.iloc[test_index], y.iloc[test_index]
        #Entrenamos el modelo
        model.fit(x_train, y_train)
        mean_score = mean_score + model.score(x_test,y_test)
        print(f"Accuracy en K = {iteration}: ", model.score(x_test,y_test))
        #Calculamos la predicción para añadirlo al histórico
        y_pred = model.predict(x_test)
        #Guardamos los datos
        y_test_hist = np.append(y_test_hist, y_test)
        y_pred_hist = np.append(y_pred_hist, y_pred)
        iteration += 1
    #Obtenemos la media de los resultados
    resultado = np.round(mean_score/n_splits,number_decimals)
    print("Resultado medio del Cross Validation: ", resultado)
    if show_confusion_matrix:
        print("-------------------------")
        calculate_confusion_matrix(y_test_hist, y_pred_hist)
    return resultado

Comenzamos la predicción de la importancia del evento a través de modelos sencillos. 

## Regresión logistica

Primero hacemos un entrenamiento base con CV con k = 5.

In [ ]:
#Entrenamos el modelo
model = LogisticRegression(max_iter=2000)
_ = train_with_CV_class(model, x, y, number_decimals=4)

En este primer modelo se puede ver como si bien entre la clase 0 y las otras dos la separación es bastante clara, sin embargo el modelo presenta más problemas para diferenciar las clases 1 y 2. Para intentar mejorar el resultado realizamos una busqueda de parámetros para ajustar mejor el modelo.

In [ ]:
start = time.time()
model = LogisticRegression(max_iter=2000)
#Definimos los parámetros a editar
c = [1, 10, 100, 1000]
param_grid = {'C': c}
#Realizamos la busqueda
search = HalvingGridSearchCV(model, param_grid, random_state=0)
search.fit(x, y)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
search.best_params_

In [ ]:
#Entrenamos el modelo
model = LogisticRegression(C=100,max_iter=2000)
result = train_with_CV_class(model, x, y, number_decimals=4)

El resultado obtenido ha sido muy similar al original y no se ha obtenido una gran mejora.

## KNN

Comenzamos las prueas con KNN realizando una busqueda del mejor valor de **k**.

In [ ]:
#Entrenamos el modelo
min_result = None
min_k = None
k_values = [5, 10, 20, 50]
for k in k_values:
    print(f"Entrenando con k (KNN) = {k}")
    model = KNeighborsClassifier(n_neighbors=k)
    result = train_with_CV_class(model, x, y, show_confusion_matrix=False)
    if min_result is None or min_result < result:
        min_result = result
        min_k = k
print(f"El mejor valor de k es {min_k}")

El mejor valor obtenido es el de **k** = 10.

In [ ]:
model = KNeighborsClassifier(n_neighbors=10)
result = train_with_CV_class(model, x, y)

En este caso se puede ver como el resultado obtenido ha mejorado al de la regresión logistica, además de que la confusión entre clases ha disminuido.

# SVC

In [ ]:
model = SVC(kernel="poly")
result = train_with_CV_class(model, x, y)

En el caso de SVC no se ha obtenido ninguna mejora respecto a los modelos anteriores.

# Random Forest

A partir de este punto continuaremos las pruebas con **ensembles**, ya que son modelos que suelen aportar una mayor precisión.

In [ ]:
#Medimos el tiempo para compararlo luego con la búsqueda de parámetros
start = time.time()
#Entrenamos el modelo
model = RandomForestClassifier(max_depth=2, random_state=0)
train_with_CV_class(model, x, y, number_decimals=4)
end = time.time()
print(f"Tiempo de entrenamiento: {end - start:.2f}s")

El resultado obtenido ha sido el peor hasta el momento, pero Random Forest puede llegar a mejorar notablemente su resultado si se ajustan adecuadamente los valores, por ello se va a realizar una búsqueda de parámetros más optimos.

In [ ]:
start = time.time()
model = RandomForestClassifier(random_state=0)
#Definimos los parámetros a editar
max_depth = [2, 5, 10, None]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 5]
param_grid = {'max_depth': max_depth, 'min_samples_split': min_samples_split, \
              'min_samples_leaf': min_samples_leaf}
#Realizamos la busqueda
search = HalvingGridSearchCV(model, param_grid, random_state=0)
search.fit(x, y)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
search.best_params_

In [ ]:
#Entrenamos el modelo
model = RandomForestClassifier(max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=0)
result = train_with_CV_class(model, x, y, number_decimals=4)

Como se puede ver el resultado ha mejorado notablemente, pasando a ser el mejor modelo obtenido hasta el momento. También se puede ver una confusión menor este las clases y que en todos las clases mayoritaria es la correcta. El resultado obtenido ha sido de una precisión del **63.2%**.

## AdaBoostClassifier (Boosting)

Continuamos probando otros modelos **ensembles**.

In [ ]:
#Medimos el tiempo para compararlo luego con la búsqueda de parámetros
start = time.time()
#Entrenamos el modelo
model = AdaBoostClassifier(random_state=0)
train_with_CV_class(model, x, y, number_decimals = 4)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
start = time.time()
model = AdaBoostClassifier(random_state=0)
#Definimos los parámetros a editar
n_estimators = [25, 50, 100, 200]
learning_rate = [0.1, 0.5, 1, 2, 5]
param_grid = {'n_estimators': n_estimators, 'learning_rate': learning_rate}
#Realizamos la busqueda
search = HalvingGridSearchCV(model, param_grid, random_state=0)
search.fit(x, y)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
search.best_params_

In [ ]:
#Entrenamos el modelo
model = AdaBoostClassifier(n_estimators=200, learning_rate=1, random_state=0)
result = train_with_CV_class(model, x, y, number_decimals=4)

En este caso el ajuste de parámetros no ha supuesto una gran mejora y el resultado obtenido es peor al de otros modelos anteriores.

# Stacking

El stacking nos permite combinar diferentes modelos para obtener uno que suele ser más preciso a cambio de una mayor complejidad. En este caso se utilizará el stacking a partir de los tres modelos que mejores resultados han proporcionado hasta el momento.

In [ ]:
from sklearn.ensemble import StackingClassifier

# Definimos los modelos base
base_models = list()
base_models.append(('lr', LogisticRegression(C=100,max_iter=2000)))
base_models.append(('knn', KNeighborsClassifier(n_neighbors=10)))
base_models.append(('rf', RandomForestClassifier(max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=0)))
# Definimos el modelo del meta learner
meta_learner = LogisticRegression(max_iter=2000)

# Definimos el stacking ensemble
model = StackingClassifier(estimators=base_models, final_estimator=meta_learner, cv=5)

In [ ]:
result = train_with_CV_class(model, x, y, number_decimals=4)

El resultado obtenido es hasta ahora el mejor, con un **63.67%** frente al **63.2%** del Random Forest.

## XGBoost

XGBoost es un modelo interesante, ya que el su paquete trae diferentes opciones de visualización y es compatible con otros tantos paquetes de Python.

In [ ]:
import warnings

warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

def train_with_CV_xgboost(model, x, y, n_splits = 5, number_decimals = 2, show_confusion_matrix = True):
    #Declaramos la función que generará las particiones del CV
    cv = KFold(n_splits=n_splits) 
    print("Se entrenará el modelo utilizando CV ")
    #Variable para acumular la media de los scores sobre los diferentes folds
    mean_score = 0.0
    iteration = 0
    y_test_hist = np.array([])
    y_pred_hist = np.array([])
    for train_index, test_index in cv.split(x):
        #Generamos el conjunto de entramiento para la partición K
        x_train, y_train = x.iloc[train_index], y.iloc[train_index]
        x_test, y_test = x.iloc[test_index], y.iloc[test_index]
        #Entrenamos el modelo
        model.fit(x_train, y_train)
        mean_score = mean_score + accuracy_score(y_test, model.predict(x_test))
        print(f"Accuracy en K = {iteration}: ", accuracy_score(y_test, model.predict(x_test)))
        #Calculamos la predicción para añadirlo al histórico
        y_pred = model.predict(x_test)
        #Guardamos los datos
        y_test_hist = np.append(y_test_hist, y_test)
        y_pred_hist = np.append(y_pred_hist, y_pred)
        iteration += 1
    #Obtenemos la media de los resultados
    resultado = np.round(mean_score/n_splits,number_decimals)
    print("Resultado medio del Cross Validation: ", resultado)
    if show_confusion_matrix:
        print("-------------------------")
        calculate_confusion_matrix(y_test_hist, y_pred_hist)

XGBoost nos permite hacer realizar clasificación tanto con **XGBClassifier** como **XGBRegressor**, si bien lo normal sería utilizar XGBClassifier, el uso de XGBRegressor permite realizar una comparación a través de la curva ROC y otras ventajas que se mostrarán más adelante. Para comprobar que ambos son igualmente eficaces se van a entrenar ambos y así se podrá observar como los valores obtenidos son muy similares.

## XGBoost (Classifier)

In [ ]:
#Medimos el tiempo para compararlo luego con la búsqueda de parámetros
start = time.time()
model = xgb.XGBClassifier(num_class=3, eval_metric="mlogloss")
train_with_CV_xgboost(model, x, y, number_decimals = 4)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
start = time.time()
model = xgb.XGBClassifier(num_class=3, eval_metric="mlogloss")
#Definimos los parámetros a editar
eta = [0.01, 0.05, 0.1, 0.3, 0.5] #Learning rate
gamma = [0, 0.5, 1]
max_depth = [5, 6, 7]
param_grid = {'eta': eta, 'gamma':gamma, 'max_depth':max_depth}
#Realizamos la busqueda
search = HalvingGridSearchCV(model, param_grid, random_state=0)
search.fit(x, y)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
search.best_params_

In [ ]:
#Entrenamos el modelo
model = xgb.XGBClassifier(objective="multi:softmax", num_class=3, eval_metric="mlogloss", eta=0.1, gamma=1, \
                        max_depth=7)
train_with_CV_xgboost(model, x, y, number_decimals=4)

La precisión obtenida con XGBClassifier ha sido de **63.47%**, siendo bastante cercana a la obtenida mediante stacking. A continuación repetiremos las pruebas con XGBRegressor.

## XGBoost (Regressor)

In [ ]:
#Medimos el tiempo para compararlo luego con la búsqueda de parámetros
start = time.time()
model = xgb.XGBRegressor(objective="multi:softmax", num_class=3, eval_metric="mlogloss")
train_with_CV_xgboost(model, x, y, number_decimals = 4)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
start = time.time()
model = xgb.XGBRegressor(objective="multi:softmax", num_class=3, eval_metric="mlogloss")
#Definimos los parámetros a editar
eta = [0.01, 0.05, 0.1, 0.3, 0.5] #Learning rate
gamma = [0, 0.5, 1]
max_depth = [5, 6, 7]
param_grid = {'eta': eta, 'gamma':gamma, 'max_depth':max_depth}
#Realizamos la busqueda
search = HalvingGridSearchCV(model, param_grid, random_state=0)
search.fit(x, y)
end = time.time()
print(f"Tiempo de búsqueda: {end - start:.2f}s")

In [ ]:
search.best_params_

In [ ]:
#Entrenamos el modelo
model = xgb.XGBRegressor(objective="multi:softmax", num_class=3, eval_metric="mlogloss", eta=0.1, gamma=1, \
                        max_depth=7)
train_with_CV_xgboost(model, x, y, number_decimals=4)

Como se puede ver, los resultados obtenidos por XGBClassifier y XGBRegressor son iguales.

# Comparando los mejores modelos

Los modelos que mejores resultados han obtenido son el modelo de Stacking, con un 63.67% de precisión, y XGBoost con un 63.47% de precisión. Para tener un elemento de comparación extra vamos a utilizar la curva ROC.

In [ ]:
def calcula_roc_data(model, x, y, n_splits=5, direct_train = False):
    cv = KFold(n_splits=n_splits)
    n_classes = max(y) + 1
    #Binarizamos la variable Y
    if not direct_train:
        y = label_binarize(y, classes=list(range(n_classes)))
    y_test_hist = np.array([])
    y_score_hist = np.array([])
    #Creamos el dataset con Y binarizada
    for train_index, test_index in cv.split(x):
        #Generamos el conjunto de entramiento para la partición K
        x_train, y_train = x.iloc[train_index], y[train_index]
        x_test, y_test = x.iloc[test_index], y[test_index]
        #Entrenamos el modelo utilizando el paradigma "One VS Rest"
        random_state = np.random.RandomState(0)
        if direct_train:
            classifier = model.fit(x_train, y_train)
            y_score = classifier.predict(x_test)
        else:
            classifier = OneVsRestClassifier(model).fit(x_train, y_train)
            y_score = classifier.decision_function(x_test)
        #Guardamos los datos en el histórico
        if not direct_train:
            y_test_hist = np.append(y_test_hist, y_test)
        else:
            y_test_hist = np.append(y_test_hist, label_binarize(y_test, classes=list(range(n_classes))))
        y_score_hist = np.append(y_score_hist, y_score)
    #Definimos los diccionarios donde guardaremos los resultados
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    # Redimensionamos los arrays
    y_test_hist = np.reshape(y_test_hist, (-1, n_classes))
    y_score_hist = np.reshape(y_score_hist, (-1, n_classes))
    # Calculamos la curva ROC curve y el area de esta para cada clase
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_hist[:, i], y_score_hist[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Calculamos el 'micro-average ROC curve' y la area ROC
    fpr["micro"], tpr["micro"], thresholds_micro = roc_curve(y_test_hist.ravel(), y_score_hist.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    return tpr, fpr, roc_auc
        
def show_curve_roc(tpr, fpr, roc_auc, model_name):
    plt.figure(figsize=(10, 8))
    lw = 2
    n_classes = max(y) + 1
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i],
                 lw=lw, label=f'Curva ROC (area = %0.2f) - Clase {i}' % roc_auc[i])
    plt.plot(fpr["micro"], tpr["micro"],
             label='Curva micro-average ROC (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='black', linestyle=':', linewidth=4)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Curva ROC para el modelo de {model_name}')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
#Definimos el modelo
model = StackingClassifier(estimators=base_models, final_estimator=meta_learner, cv=5)
tpr, fpr, roc_auc = calcula_roc_data(model, x, y)
show_curve_roc(tpr, fpr, roc_auc, "regresión polinómica con parámetros ajustados")

In [ ]:
model = xgb.XGBRegressor(objective="multi:softprob", num_class=3, eval_metric="mlogloss")
tpr, fpr, roc_auc = calcula_roc_data(model, x, y, direct_train=True)
show_curve_roc(tpr, fpr, roc_auc, "regresión polinómica con parámetros ajustados")
model.fit(x, y)

Como se observar, ambos modelos presentan una precisión muy similar, obtenido el modelo de Stacking una área bajo la curva de 0.84, frente a XGBoost, que obtiene un área bajo la curva de 0.83.

Si bien la precisión del modelo de Stacking es ligeramente superior el modelo de XGBoost presenta una ventaja, ya que este tiene una mayor explicabilidad, por lo que es preferible a pesar de suponer una pequeña disminución de precisión.

# Análisis de características

El primer elemento que vamos a explorar con XGBoost es la importancia de las diferentes features a la hora de obtener su resultado.

In [ ]:
model = xgb.XGBRegressor(objective="multi:softmax", num_class=3, eval_metric="mlogloss", eta=0.1, gamma=1, \
                        max_depth=7).fit(x, y)

In [ ]:
xgb.plot_importance(model)
plt.show()

Como se puede ver las variables más importantes a la hora de determinar la importancia del incidente es la población y la densidad de población del lugar donde tiene lugar, seguido del número de implicados y si existe el conocimiento de que haya un arma involucrada.

Esto nos aporta información, pero se encuentra limitada, para aumentar la explicabilidad del modelo vamos a hacer uso de [SHAP](https://github.com/slundberg/shap), que es una técnica aplicable a XGBoost para obtener un mayor conocimiento sobre los elementos que han dado lugar a la predicción del modelo.

Primero vamos a comenzar explorando la relevancia de las features en cada clase.

In [ ]:
shap.initjs()
model = xgb.XGBClassifier(num_class=3, eval_metric="mlogloss", eta=0.1, gamma=1, max_depth=7).fit(x, y)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x,approximate=True)

## Clase 0: Importancia baja

In [ ]:
shap.summary_plot(shap_values[0], x)

En este primer caso se pueden observar diferentes elementos:
- La existencia de adultos involucrados hace menos probable que una incidencia tenga un importancia baja.
- La densidad de población baja incentiva una gravedad menor del incidente.
- El area 0, la zona norte de California (San Francisco, San José y Sacramento) suele tener incidentes de mayor gravedad que la zona sur (Los Ángeles).
- Curiosamente, el hecho de que se haya reportado que hay un arma robada suele implicar que el incidente suele tener menor gravedad.

## Clase 1: Importancia media

In [ ]:
shap.summary_plot(shap_values[1], x)

En el caso de los incidentes de importancia/gravedad media la relevancia de los diferentes elementos es menos marcada, pero entre ellos destacan:
- Un mayor número de implicados suele conllevar mayor posibilidades de que sea un incidente de gravedad media aunque hay cierto ruido.
- Una mayor densidad de población suele provocar una tendencia hacia una importancia media.
- La presencia de adolescentes y niños suele llevar a una mayor probabilidad de que sea un incidente de prioridad alta. 

## Clase 2: Importancia alta

In [ ]:
shap.summary_plot(shap_values[2], x)

En el caso de los incidentes de prioridad alta hay diferentes elementos a destacar:
- Un mayor número de implicados tiende a dar lugar a que la importancia del evento sea alta.
- El hecho de que haya un arma robada reduce la posibilidad de que sea un evento de prioridad alta.
- La existencia de un sospechoso reduce la probabilidades de que la incidencia sea de gravedad alta.

# Escala única

Al ser realmente las clases creadas una escala ascendente de importancia/gravedad del incidente esto nos permite también realizar una escala de importancia, donde a mayor valor mayor importancia, permitiendonos utilizar una regresión y así observar la influencia de las características del indicente de forma global y en una escala única y continua.

Vamos a comenzar a analizar casos a nivel individual.

In [ ]:
model = xgb.XGBRegressor(eval_metric="mlogloss", eta=0.1, gamma=1, max_depth=7).fit(x, y)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(x)

In [ ]:
data.head(20)

## Analizando caso de importancia baja

In [ ]:
shap.plots.waterfall(shap_values[7])

In [ ]:
shap.plots.force(shap_values[7], matplotlib=True)

Se puede observar que los dos elementos más relevantes han sido la existencia de un sospechoso y un número bajo de implicados.

## Analizando caso de importancia media

In [ ]:
shap.plots.waterfall(shap_values[2])

In [ ]:
shap.plots.force(shap_values[2], matplotlib=True)

En este caso, al pertenecer a un caso de importancia media, se pueden ver que los elementos más relevantes tanto aumentan como disminuyen la gravedad, siendo los elementos que más disminuyen la gravedad el hecho de que exista un sospechoso y sea una densidad relativamente baja, mientras que aumenta la gravedad que el número de implicados sea mayor que en el caso anterior, que haya una mujer involucrada, que tiene lugar en la zona norte de California y la presencia de un adulto.

## Analizando caso de importancia alta

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.force(shap_values[0], matplotlib=True)

En este caso se puede observar la gran relevancia que le da a la existencia de un sospechoso el modelo, pudiendo ver como elementos como el no haberse realizado en la zona norte de California y una densidad de población baja disminuyen la importancia de la incidencia, pero en una cantidad muy pequeña comparada a la importancia de la no existencia de un sospechoso.

# Análisis global de la importancia de un incidente

Al igual que en el caso anterior se ha analizado la influencia global de las variables por cada clase, en este caso se va a analizar todos los casos conjuntamente, representando la importancia del incidente de forma continua. 

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(x)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=20)

Al analizar todas los casos de forma continua respecto a su severidad se pueden observar las tendencias de forma más clara, ya que en los casos de severidad media se podía apreciar una menor determinación de la relevancia de las características. En este caso se pueden observar los siguientes elementos:
- La existencia de un sospechoso disminuye la gravedad del incidente, llegando a influir en gran cantidad la no existencia de un sospechoso a la hora de aumentar la gravedad de un incidente.
- A mayor número de implicados, mayor importancia del incidente.
- A mayor densidad de población, mayor probabilidad de que incidente sea más grave.
- La zona norte de California (area_0) tiende a tener incidentes más graves.
- En general, la presencia de mujeres da lugar a incidentes más graves. 
- El conocimiento de que existe un arma robada suele disminuir la importancia del incidente.
- La presencia de adolescente suele provocar eventos más graves.
- El resto de variables suelen aumentar o disminuir la gravedad del incidente en base a otras características, o directamente no suelen tener relevancia.

El hecho de que varias variables referentes a los clusters tengan relevancia tan baja posiblemente se deba a que su información se encuentra contenida en otras variables y el modelo obtenga esta información de esas variables.

# Conclusiones

Se ha conseguido obtener un modelo que tiene una buena precisión prediciendo cuando un evento tendrá una importancia/gravedad baja, pero presenta mayores problemas para diferenciar entre eventos de gravedad media y alta. Sin embargo, esto es un buen resultado, ya que en caso de agrupar los eventos de prioridad media y alta se podría distinguir con bastante precisión entre incidentes prioritarios y no prioritarios, siendo de ayudar a la hora de dar prioridad a los diferentes eventos cuando existiese una falta de recursos.

También es de destacar que se ha obtenido una buena explicabilidad del modelo, lo que permitiría al usuario entender mejor los resultados que el modelo le está proporcionando, para así darle mayor confianza sobre la predicción obtenida.

# Líneas futuras de trabajo

El resultado obtenido ha sido satisfactorio pero hay diversos elementos en los que se podría profundizar:
- **Enriquecimiento de los datos**: Hay diversos datos a nivel socioeconómico que se podrían utilizar para enriquecer más los datos, ya que aparte de los datos de población, densidad de población y área añadidos a cada caso se podría utilizar, por ejemplo, el PIB per cápita de la zona.
- **Combinación de categorías**: El principal problema que presenta el modelo es el de diferenciar entre los casos de importancia media e importancia alta, por ello se podría contemplar la idea de reformular el problema como la división de los casos entre prioritarios y no prioritarios.